In [13]:
import pandas as pd

In [14]:
data = pd.read_csv('test/cities.csv')

# Randomized the arrays to assert no possible order is maintained
data_rand = data.sample(frac=1)
assert data_rand.to_dict('records') != data.to_dict('records')

display(data.info())
display(data.sample(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128 entries, 0 to 127
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   LatD    128 non-null    int64 
 1   LatM    128 non-null    int64 
 2   LatS    128 non-null    int64 
 3   NS      128 non-null    object
 4   LonD    128 non-null    int64 
 5   LonM    128 non-null    int64 
 6   LonS    128 non-null    int64 
 7   EW      128 non-null    object
 8   City    128 non-null    object
 9   State   128 non-null    object
dtypes: int64(6), object(4)
memory usage: 10.1+ KB


None

,LatD,LatM,LatS,NS,LonD,LonM,LonS,EW,City,State
95,31,27,35,N,100,26,24,W,San Angelo,TX
102,44,57,0,N,93,5,59,W,Saint Paul,MN
49,41,39,0,N,83,32,24,W,Toledo,OH
63,38,9,0,N,79,4,11,W,Staunton,VA
65,37,13,12,N,93,17,24,W,Springfield,MO


In [15]:
def test_SORT_COLUMN():
    from app.operations import SortColumn
    assert SortColumn(column_name='LatD')(data)['LatD'].to_list() == sorted(data['LatD'].to_list())

test_SORT_COLUMN()

In [16]:
def test_REINDEX_COLUMN():
    from app.operations import ReindexColumn
    assert ReindexColumn(column_name='City')(data).index.name == 'City'

test_REINDEX_COLUMN()

In [17]:
def test_REMOVE_DUPLICATES_VALUES_COLUMN():
    from app.operations import RemoveDuplicateValuesColumn
    assert list(sorted(RemoveDuplicateValuesColumn(column_names=['State'])(data)['State'].to_list())) == list(sorted(set(data['State'].to_list())))

test_REMOVE_DUPLICATES_VALUES_COLUMN()

In [18]:
import requests as rq

url = "http://0.0.0.0:8000/file"

body = {
    "file": {
        "alias": 'test_file',
        "data": data.to_dict("records"),
    },
    "operations": [
        {
            "code": "RENAME_COLUMN",
            "attributes": {
                "column_names": ["State", "City"],
                "new_names": ["STATE", "CITY"]
            }
        },
        {
            "code": "SORT_COLUMN",
            "attributes": {
                "column_name": "CITY",
                "ascending": True
            }
        }
    ],
}

response = rq.post(url, json=body)
print(response.status_code)

response_file = response.json()
response_file_df = pd.DataFrame(response_file['data'])
response_file_df.head()

200


,LatD,LatM,LatS,NS,LonD,LonM,LonS,EW,CITY,STATE
0,41,9,35,N,81,14,23,W,Ravenna,OH
1,40,19,48,N,75,55,48,W,Reading,PA
2,40,10,48,N,122,14,23,W,Red Bluff,CA
3,50,25,11,N,104,39,0,W,Regina,SA
4,39,31,12,N,119,48,35,W,Reno,NV


In [19]:
data.sort_values(by='City', ascending=True).head()

,LatD,LatM,LatS,NS,LonD,LonM,LonS,EW,City,State
127,41,9,35,N,81,14,23,W,Ravenna,OH
126,40,19,48,N,75,55,48,W,Reading,PA
125,40,10,48,N,122,14,23,W,Red Bluff,CA
124,50,25,11,N,104,39,0,W,Regina,SA
123,39,31,12,N,119,48,35,W,Reno,NV
